In [200]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV,train_test_split,StratifiedKFold
from sklearn.cluster import KMeans
from sklearn.svm  import SVC
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (accuracy_score, balanced_accuracy_score,f1_score, classification_report, confusion_matrix)

In [68]:
df = pd.read_csv("Data/cirrhosis.csv")
df.drop(df.columns[0],axis=1,inplace=True)
df = df[0:312]
df


,N_Days,Status,Drug,Age,Sex,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Cholesterol,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,400,D,D-penicillamine,21464,F,Y,Y,Y,Y,14.5,261.0,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,4500,C,D-penicillamine,20617,F,N,Y,Y,N,1.1,302.0,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,1012,D,D-penicillamine,25594,M,N,N,N,S,1.4,176.0,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,1925,D,D-penicillamine,19994,F,N,Y,Y,S,1.8,244.0,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,1504,CL,Placebo,13918,F,N,Y,Y,N,3.4,279.0,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,1153,C,D-penicillamine,22347,F,N,Y,N,N,0.4,246.0,3.58,24.0,797.0,91.00,113.0,288.0,10.4,2.0
308,994,C,Placebo,21294,F,N,N,N,N,0.4,260.0,2.75,41.0,1166.0,70.00,82.0,231.0,10.8,2.0
309,939,C,D-penicillamine,22767,F,N,N,N,N,1.7,434.0,3.35,39.0,1713.0,171.00,100.0,234.0,10.2,2.0
310,839,C,D-penicillamine,13879,F,N,N,N,N,2.0,247.0,3.16,69.0,1050.0,117.00,88.0,335.0,10.5,2.0


In [ ]:
# df[df.columns[2]].fillna("Other",inplace=True)
for i in df.columns:
    if df[i].isna().any()==True:
        df.fillna({i: df[i].mean()},inplace=True)  #I have written this line knowing that categorical variables no longer have any NAN values
df["Age"] = round(df["Age"]/365)

In [ ]:
lists = [
df.groupby("Stage")["N_Days"].mean(),
df.groupby("Stage")["Age"].mean(),
df.groupby("Stage")[df.columns[9]].mean(),
df.groupby("Stage")[df.columns[10]].mean(),
df.groupby("Stage")[df.columns[11]].mean(),
df.groupby("Stage")[df.columns[12]].mean(),
df.groupby("Stage")[df.columns[13]].mean(),
df.groupby("Stage")[df.columns[14]].mean(),
df.groupby("Stage")[df.columns[15]].mean(),
df.groupby("Stage")[df.columns[16]].mean(),
df.groupby("Stage")[df.columns[17]].mean()]

print(pd.concat(lists,axis=1))
#Platelets,Age,tryglycerides,SGOT,alkaline phosphatase,Copper,albumin,Bilirubin,prothrombin,N_Days

            N_Days        Age  Bilirubin  Cholesterol   Albumin      Copper  \
Stage                                                                         
1.0    2940.250000  46.375000   1.075000   286.908231  3.739375   62.812500   
2.0    2401.940299  49.298507   2.128358   354.642737  3.627612   68.472364   
3.0    2140.866667  48.350000   2.900000   413.921273  3.634000   92.075000   
4.0    1478.045872  52.954128   4.661468   341.882091  3.296147  126.831637   

          Alk_Phos        SGOT  Tryglicerides   Platelets  Prothrombin  
Stage                                                                   
1.0    1694.212500   84.460000      97.194149  277.433442    10.868750  
2.0    1816.907463  115.562388     114.913623  296.700523    10.344776  
3.0    2082.128333  124.326750     130.382624  270.415584    10.497500  
4.0    2017.366972  130.498440     128.503026  228.954128    11.189908  


In [71]:
print(pd.crosstab(df["Drug"],df["Stage"],normalize='index') ,'\n')
print(pd.crosstab(df["Sex"],df["Stage"],normalize='index'),'\n' )
print(pd.crosstab(df["Status"],df["Stage"],normalize='index'),'\n' )
print(pd.crosstab(df[df.columns[5]],df["Stage"],normalize='index'),'\n')
print(pd.crosstab(df[df.columns[6]],df["Stage"],normalize='index'),'\n' )
print(pd.crosstab(df[df.columns[7]],df["Stage"],normalize='index'),'\n' )
print(pd.crosstab(df[df.columns[8]],df["Stage"],normalize='index'),'\n' )

# Status,Ascites,Hepatomehgaly,Spiders,Edema
#Margins = True for displaying row and column total 
#normalize = 'index' for showing row relative frequencies

Stage                 1.0       2.0       3.0       4.0
Drug                                                   
D-penicillamine  0.075949  0.221519  0.354430  0.348101
Placebo          0.025974  0.207792  0.415584  0.350649 

Stage       1.0       2.0       3.0       4.0
Sex                                          
F      0.047101  0.221014  0.391304  0.340580
M      0.083333  0.166667  0.333333  0.416667 

Stage        1.0       2.0       3.0       4.0
Status                                        
C       0.089286  0.285714  0.410714  0.214286
CL      0.000000  0.157895  0.421053  0.421053
D       0.008000  0.128000  0.344000  0.520000 

Stage         1.0       2.0       3.0       4.0
Ascites                                        
N        0.055556  0.225694  0.413194  0.305556
Y        0.000000  0.083333  0.041667  0.875000 

Stage              1.0       2.0       3.0       4.0
Hepatomegaly                                        
N             0.105263  0.315789  0.440789  0.13815

In [72]:
df1 = df.drop(["Sex","Drug","Cholesterol"],axis=1)   #filtering the necessary features
df1

,N_Days,Status,Age,Ascites,Hepatomegaly,Spiders,Edema,Bilirubin,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin,Stage
0,400,D,59.0,Y,Y,Y,Y,14.5,2.60,156.0,1718.0,137.95,172.0,190.0,12.2,4.0
1,4500,C,56.0,N,Y,Y,N,1.1,4.14,54.0,7394.8,113.52,88.0,221.0,10.6,3.0
2,1012,D,70.0,N,N,N,S,1.4,3.48,210.0,516.0,96.10,55.0,151.0,12.0,4.0
3,1925,D,55.0,N,Y,Y,S,1.8,2.54,64.0,6121.8,60.63,92.0,183.0,10.3,4.0
4,1504,CL,38.0,N,Y,Y,N,3.4,3.53,143.0,671.0,113.15,72.0,136.0,10.9,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307,1153,C,61.0,N,Y,N,N,0.4,3.58,24.0,797.0,91.00,113.0,288.0,10.4,2.0
308,994,C,58.0,N,N,N,N,0.4,2.75,41.0,1166.0,70.00,82.0,231.0,10.8,2.0
309,939,C,62.0,N,N,N,N,1.7,3.35,39.0,1713.0,171.00,100.0,234.0,10.2,2.0
310,839,C,38.0,N,N,N,N,2.0,3.16,69.0,1050.0,117.00,88.0,335.0,10.5,2.0


In [118]:
"""While StandardScaler can be a precursor to methods that might leverage standardized data for outlier identification (like Z-score thresholding), 
it is not a method for outlier detection in itself and is sensitive to the very outliers one might be trying to detect. """

df2 = df1.select_dtypes(["int","float"]).drop("Stage",axis=1)
# min,max = [],[]
# for i in df2.columns:
#     min.append(df2[i].quantile(0.001))
#     max.append(df2[i].quantile(0.999))
mins = df2.quantile(0.001)
maxs = df2.quantile(0.999)
mask = ((df2[df2.columns] > mins ) & (df2[df2.columns] < maxs)).all(axis=1)
df2 = df2.loc[mask]
df2
   #We can use both df.loc[..] and df[condition] but loc[] is preferred for boolean indexing




,N_Days,Age,Bilirubin,Albumin,Copper,Alk_Phos,SGOT,Tryglicerides,Platelets,Prothrombin
0,400,59.0,14.5,2.60,156.0,1718.0,137.95,172.0,190.0,12.2
1,4500,56.0,1.1,4.14,54.0,7394.8,113.52,88.0,221.0,10.6
2,1012,70.0,1.4,3.48,210.0,516.0,96.10,55.0,151.0,12.0
3,1925,55.0,1.8,2.54,64.0,6121.8,60.63,92.0,183.0,10.3
4,1504,38.0,3.4,3.53,143.0,671.0,113.15,72.0,136.0,10.9
...,...,...,...,...,...,...,...,...,...,...
307,1153,61.0,0.4,3.58,24.0,797.0,91.00,113.0,288.0,10.4
308,994,58.0,0.4,2.75,41.0,1166.0,70.00,82.0,231.0,10.8
309,939,62.0,1.7,3.35,39.0,1713.0,171.00,100.0,234.0,10.2
310,839,38.0,2.0,3.16,69.0,1050.0,117.00,88.0,335.0,10.5


In [192]:
dummies = pd.get_dummies(df1.select_dtypes(["object"]), drop_first=True).loc[df2.index]
X = pd.concat([dummies, df2], axis=1)
Y = df1["Stage"].loc[df2.index]

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

numeric_cols = df2.columns.tolist()   # order preserved from df2
cat_cols = [c for c in X.columns if c not in numeric_cols]   #There is no String in X since all converted to dummies

st = StandardScaler()

X_train_num = pd.DataFrame(st.fit_transform(X_train[numeric_cols]), columns=numeric_cols, index=X_train.index)  
X_test_num  = pd.DataFrame(st.transform(X_test[numeric_cols]),  columns=numeric_cols, index=X_test.index)

"""We are standardizing the training and testing data using statistical values of training data only. So, the statistics of test data remains hidden making it unbiased."""

# Joining categorical and numerical columns and indexing them appropriately for both test and train data
X_train_scaled = pd.concat([X_train[cat_cols].reset_index(drop=True),X_train_num.reset_index(drop=True)], axis=1)
X_train_scaled.index = X_train.index  # restore original index
X_test_scaled = pd.concat([X_test[cat_cols].reset_index(drop=True),X_test_num.reset_index(drop=True)], axis=1)
X_test_scaled.index = X_test.index

a = [7,8,10,15]
# df3 = pd.DataFrame(st.fit_transform(df2),columns=df2.columns,index=df2.index)
# mask = (np.abs(df3[df3.columns[a]]) < 3*(df3[df3.columns[a]].std())).all(axis = 1)  

mask = (np.abs(X_train_scaled[X_train_scaled.columns[a]]) < 3 * X_train_scaled[X_train_scaled.columns[a]].std()).all(axis=1)

X_train_final = X_train_scaled.loc[mask]
y_train_final = y_train.loc[X_train_final.index]
"""df3.columns[a] → selects only the columns at positions 0, 1, 3, 8 in df3.
(np.abs(... ) < 3*(...)) → produces a Boolean DataFrame for those 4 columns only.
.all(axis=1) → for each row, checks if all 4 of those selected columns satisfy the condition."""

# plt.hist(X_train_scaled[X_train_scaled.columns[10]])
# 7,8,10,15 columns are normally distributed
# df3 = df3.loc[mask]

'df3.columns[a] → selects only the columns at positions 0, 1, 3, 8 in df3.\n(np.abs(... ) < 3*(...)) → produces a Boolean DataFrame for those 4 columns only.\n.all(axis=1) → for each row, checks if all 4 of those selected columns satisfy the condition.'

In [206]:
models = {"LogisticRegression":{"model": LogisticRegression(solver="saga", max_iter=5000, random_state=36),
                                "param":{"C":[0.01,0.1,1.0,5.0],"penalty":['l1','l2']}},
         "SupportVector"      :{"model":SVC(gamma="scale",probability=False),
                                "param":{"kernel":['linear','rbf'],"C":[.1,1,10]}},
         "RandomForest"       :{"model":RandomForestClassifier(),
                                "param":{"n_estimators":[100,200]}},
         "DecisionTree"       :{"model":DecisionTreeClassifier(),
                                "param":{}},
         
}
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring_for_grid = "balanced_accuracy"   # better for class imbalance

b=[]
for i,j in models.items():
    grid = GridSearchCV(j["model"],param_grid=j["param"],cv=cv,return_train_score=False)
    grid.fit(X_train_final,y_train_final)
    best = grd.best_estimator_
    y_pred = best.predict(X_test_scaled)

    acc = accuracy_score(y_test, y_pred)
    bal_acc = balanced_accuracy_score(y_test, y_pred)
    f1_macro = f1_score(y_test, y_pred, average="macro")

    b.append({
        "model":i,
        "Best_params":best_params,
        "Score":best_cv_score,
        "test_accuracy": acc,
        "test_balanced_accuracy":bal_acc,
        "test_f1_macro": f1_macro
    })




k = pd.DataFrame(b).sort_values("test_balanced_accuracy", ascending=False)


In [207]:
# 1. Find best model by balanced accuracy (or whichever metric you want)
best_row = k.loc[k["test_balanced_accuracy"].idxmax()]
print("Best model:", best_row["model"])

# 2. Refit that model on training set to get estimator
best_model = models[best_row["model"]]["model"].set_params(**best_row["Best_params"])
best_model.fit(X_train_final, y_train_final)

# 3. Predictions
y_pred = best_model.predict(X_test_scaled)

# 4. Reports
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Best model: LogisticRegression

Classification Report:
               precision    recall  f1-score   support

         1.0       0.00      0.00      0.00         3
         2.0       0.23      0.25      0.24        12
         3.0       0.52      0.61      0.56        23
         4.0       0.78      0.67      0.72        21

    accuracy                           0.53        59
   macro avg       0.38      0.38      0.38        59
weighted avg       0.53      0.53      0.52        59


Confusion Matrix:
 [[ 0  2  1  0]
 [ 0  3  7  2]
 [ 1  6 14  2]
 [ 0  2  5 14]]
